# Dose response curve fitting examples
We used previously established EC50s for OT-1 peptides in the model, based on regular T cells. Here, we provide a dose response experiment to show that CAR OT-1 T cells have essentially identical dose response curves of TCR antigens compared to the TCR-only T cells. 

In [1]:
import matplotlib.pyplot as plt
import matplotlib as mpl
import numpy as np
import pandas as pd
import scipy as sp
from scipy import optimize, stats
import seaborn as sns
sns.set_context('talk')
idx = pd.IndexSlice
import sys, os

In [2]:
do_save_outputs = False
do_save_plots = False

In [3]:
# Relevant folders
root_dir = "../"
if root_dir not in sys.path:
    sys.path.insert(1, root_dir)
data_dir = os.path.join(root_dir, 'data', 'dose_response')
fig_dir = os.path.join(root_dir, 'figures', 'dose_response')

# Import data

### Important: this dose response is with mock-transduced T cells!
So blast T cells, T cells that go through the same process as our CD19 CAR-T cells but without transduction of a CAR.  

This means that the dose response will be very sharp, and that V4, G4 might cause very little activation. Moreover, the absolute pulse concentration EC50s might be smaller than with naive T cells. However, relative to NB4, EC50s of other peptides should line up still (except we might have problems with V4, G4). 

This is in part why we used previous EC50s in `potencies_df_2021.json`: they were established in the same kind of naive T cells that we use in the TCR/TCR antagonism experiments. 

In [ ]:
df = pd.read_hdf(os.path.join(data_dir, "ot1cytEC50Df.hdf"), key="df")
# Rename Concentration_M
new_names = list(df.index.names)
new_names[new_names.index("Concentration_M")] = "Concentration (M)"
df.index = df.index.set_names(new_names)

df

In [ ]:
plotDf = df.reset_index()
log10conc_lbl = r"$\log_{10}$ Concentration (M)"
plotDf[log10conc_lbl] = np.log10(plotDf["Concentration (M)"])
g = sns.relplot(data=plotDf, x=r"$\log_{10}$ Concentration (M)", y=24.0, row="Cytokine", 
            col="CAR", style="Spleen", hue="Peptide", kind="line")
for ax in g.axes.flat:
    ax.set_yscale("log")
plt.show()
plt.close()

## Data cleanup
- Remove cytokines without significant signal. Basically, keep IL-2, TNF, IFNg. 
- Remove mispipette accident for Q4 at 1 pM. 
- Clip IFNg data at value for None, highest concentration. Since all "None" conditions were just unpulsed tumors, they should all be identical, yet there is a systematic bias in the background noise, which increases with pulse concentration, especially in IFNg. This might be due to layout on the plate. Anyways, to prevent fitting Hill curves on this low ramp, I clip IFNg to the highest baseline value available; everything below is noise and should be treated as lower LOD. 

In [ ]:
# Detectable cytokines
df_clean = df.loc[df.index.isin(["IFNg", "IL-2", "TNFa"], level="Cytokine")]

# Mispipette of Q4 1 pM
sizes_before_after = [df_clean.shape[0]]
df_clean = df_clean.query("Peptide != 'Q4' or Concentration != '1pM'").sort_index()
sizes_before_after.append(df_clean.shape[0])
print("Size before: {0[0]}\nSize after: {0[1]}".format(sizes_before_after))
assert df.loc[(["IFNg", "IL-2", "TNFa"], slice(None), slice(None), "Q4", "1pM")].shape[0] == (sizes_before_after[0] - sizes_before_after[1])

# Clip IFNg. Baseline from Mock CARs
baseline_ifng = df_clean.loc[("IFNg", "Mock", slice(None), "None", "10nM"), :].max().max()
df_clean.loc["IFNg", :] = df_clean.loc["IFNg",:].clip(lower=baseline_ifng).values

In [ ]:
#@title Cleaned up data
plotDf = df_clean.reset_index()
log10conc_lbl = r"$\log_{10}$ Concentration (M)"
plotDf[log10conc_lbl] = np.log10(plotDf["Concentration (M)"])
g = sns.relplot(data=plotDf, x=r"$\log_{10}$ Concentration (M)", y=24.0, row="Cytokine", 
            col="CAR", style="Spleen", hue="Peptide", kind="line")
for ax in g.axes.flat:
    ax.set_yscale("log")
plt.show()
plt.close()

# Hill fits

I fit Hill function in log-log scale. In other words, for $x = \log_{10} L$ with $L$ the pulse concentration, and $y = \log_{10} C$, with $C$ the cytokine concentration, I fit

$$ y = y_0 \frac{x^h}{x^h + k^h} + b $$

where $y_0$ is the amplitude, $h$ is the Hill exponent, $k$ is the EC50, and $b$ is the background. 

In [8]:
from utils.fitting import (
    r_squared, 
    find_bounds_on_min
)

# Fit each spleen separately
This gives statistics on the resulting EC50s, an assessment of the quality of fit, etc. 

Also better biologically speaking: 3 spleens are 3 separate systems responding to the different antigens. 
The response of each could be different in principle (the goal is to minimize that difference by controlling external factors in the lab though), so each should have its own fit. 

Also, avoids having to compute error bars on the fits themselves. We compute error bars on the extracted EC50s by comparing individual line fits. 


In [9]:
def hillFunction4p(x, params):
    amplitude = params[0]
    ec50 = params[1]
    background = params[2]
    hill_k = params[3]
    return amplitude * x**hill_k/(ec50**hill_k + x**hill_k) + background


# Function which computes the vector of residuals, with the signature fun(x, *args, **kwargs)
# Fitting Hill in log-log. 
def cost_fit_hill4p(hill_pms, xpts, ypts, yerr, p0, reg_rate=0.2):
    """ p0: value around which to regularize each param. L1 regularization ="""
    # Compute Hill function at xpts
    y_fit = hillFunction4p(xpts, hill_pms)
    resids = (ypts - y_fit) / yerr
    # Add in L1 regularization
    regul = np.sqrt(reg_rate*np.abs(hill_pms - p0))
    resids = np.concatenate([resids, regul])
    return resids


#@title Function to apply to each CAR type and each cytokine
# Fit Hill exponent too as grid search will be too slow for my patience otherwise
def fit_hill_cyto_each_peptide_replicate(ser, hill_k_bounds=(4, 16), conc_lbl=None, reg_rate=0.2):
    """ Input should be the log of cytokines. 
    Grid search over all integer Hill k. Consider from 1 up to max_hill. 
    """
    hill_params_dict = {}
    if conc_lbl is None:
        conc_lbl = "Concentration"
    min_conc = np.min(ser.index.get_level_values(conc_lbl).values)
    concentrations = np.log10(ser.index.get_level_values(conc_lbl).values / min_conc)
    conc_ser = pd.Series(concentrations, index=ser.index, name="log10 Concentration")
    df_fit_data = pd.concat({"log10 Cytokine":ser, "log10 Concentration":conc_ser}, axis=1)
    for peptide in pd.unique(ser.index.get_level_values('Peptide')):
        # Loop on all subdivisions other than peptide concentration, which is the x axis
        pep_ser = ser.xs(peptide, level="Peptide")
        pep_conc_ser = conc_ser.xs(peptide, level="Peptide")
        drop_lvls = [a for a in pep_ser.index.names if a.startswith("Concentr")]
        loop_idx = pep_ser.droplevel(drop_lvls).index.unique()
        # Reorder levels
        pep_ser = pep_ser.reorder_levels(list(loop_idx.names) + drop_lvls)
        pep_conc_ser = pep_conc_ser.reorder_levels(list(loop_idx.names) + drop_lvls)
        for ky in loop_idx:
            y_ser = pep_ser.loc[ky]
            x_ser = pep_conc_ser.loc[ky]
            data = y_ser.values.flatten()
            err = 1.0

            # Finding bounds on the parameter values
            # Background limits
            min_back, max_back = find_bounds_on_min(data)

            # Concentration limits: always lower=0 because rescaled
            max_conc = np.max(x_ser)*2
            if max_conc == 0:
                max_conc = 6
        
            min_conc2 = np.min(x_ser)
            data_replicates = ser.xs(peptide, level="Peptide")
            max_amplitude = (np.max(data) - np.min(data))*2.0
            if max_amplitude == 0.0:
                max_amplitude = 0.01
            lowerbounds = np.asarray([0.0, 0, min_back, hill_k_bounds[0]])
            upperbounds = np.asarray([max_amplitude, max_conc, max_back, hill_k_bounds[1]])
            # Constrain amplitude to max value recorded for that cytokine. 
            # Assume increasing further concentration would not increase further plateau level

            # Try without bounds for now
            # Params: amplitude, ec50, background, k
            regul_p0 = np.zeros(4)
            # Regularize amplitude to be at the observed amplitude, roughly
            regul_p0[0] = 0.5 * (upperbounds[0] - lowerbounds[0])
            # Regularize EC50 to be as large as possible. 
            regul_p0[1] = np.max(x_ser) + 2
            # And Hill exponent as small as possible
            regul_p0[3] = hill_k_bounds[0]
            init_p0 = (lowerbounds + upperbounds) / 2
            # Cost args: xpts, ypts, yerr, p0]
            cost_args = (x_ser, data, err, regul_p0)
            cost_kwargs = {"reg_rate":reg_rate}
            
            result = sp.optimize.least_squares(cost_fit_hill4p, init_p0, 
                        method="trf", args=cost_args,
                        kwargs=cost_kwargs,
                        bounds=[lowerbounds, upperbounds],
                )

            popt = result.x
            r2 = round(r_squared(x_ser, data, hillFunction4p, popt), 3)
            
            key = (peptide,) + tuple(ky)
            hill_params_dict[key] = pd.Series(list(popt), name="Parameters",
                index=["amplitude", "ec50", "background", "hill_power"])
            hill_params_dict[key]["rsquared"] = r2
    lvl_names = ["Peptide"] + list(loop_idx.names) + ["Parameters"]
    df_params = pd.concat(hill_params_dict, names=lvl_names, axis=0)
    df_params = df_params.unstack("Parameters")
    conc_names = [a for a in df_fit_data.index.names if a.startswith("Concentration")]
    df_fit_data = df_fit_data.reorder_levels(lvl_names[:-1] + conc_names)
    return df_params, df_fit_data

In [10]:
#@title Test on a condition that seemed problematic
df_fit = np.log10(df_clean / df_clean.groupby("Cytokine").min()).sort_index()
min_hill_k, max_hill_k = 4, 16
fit_res,  df_fit_data = fit_hill_cyto_each_peptide_replicate(df_fit.loc[("IL-2", "CAR_Mut"), 24.0], 
              hill_k_bounds=(min_hill_k, max_hill_k), conc_lbl="Concentration (M)", reg_rate=0.02)

In [ ]:
# Plot the test fits. 
fig, ax = plt.subplots()
pep_order = ["N4", "A2", "Q4", "T4", "V4", "G4", "E1", "None"]
pep_order = [pep for pep in pep_order if pep in fit_res.index.get_level_values("Peptide").unique()]
palette = sns.color_palette(n_colors=len(pep_order))
styles = ["-", "--", ":"]
df_data = df_fit.loc[("IL-2", "CAR_Mut"), 24.0]
for i, pep in enumerate(pep_order):
    fit_res_pep = fit_res.xs(pep, level="Peptide").sort_index()
    df_fit_pep = df_fit_data.xs(pep, level="Peptide").sort_index()
    for j, rep in enumerate(fit_res_pep.index.get_level_values("Spleen").unique()):
        x = df_fit_pep.loc[rep, "log10 Concentration"].values
        xfit = np.linspace(x.min(), x.max(), 201)
        y = df_fit_pep.loc[rep, "log10 Cytokine"].values
        pms = fit_res_pep.loc[rep, "amplitude":"hill_power"].values
        yfit = hillFunction4p(xfit, pms)
        ax.plot(x, y, marker="o", mfc=palette[i], mec=palette[i], ls="none")
        lbl = pep if j == 0 else None
        ax.plot(xfit, yfit, color=palette[i], label=lbl, ls=styles[j])
ax.legend(loc="upper left", bbox_to_anchor=(1, 1))
ax.set(xlabel=r"$\log_{10}$ Concentration", ylabel=r"$\log_{10}$ Cytokine")
plt.show()
plt.close()

In [ ]:
#@title Fit all dose response curves
df_fit = np.log10(df_clean / df_clean.groupby("Cytokine").min()).sort_index()
min_hill_k, max_hill_k = 4, 16
# Different reg. rate for each parameter. Constrain amplitude a lot. 
# amplitude, ec50, background
regular_rate = 0.01

all_fit_results4p = {}
data_to_plot4p = {}
for cyt in df_clean.index.get_level_values("Cytokine").unique():
    for car in df_clean.index.get_level_values("CAR").unique():
        print("Fitting {}, {}".format(cyt, car))
        fit_res, df_all_stats = fit_hill_cyto_each_peptide_replicate(df_fit.loc[(cyt, car), 24.0], 
              hill_k_bounds=(min_hill_k, max_hill_k), conc_lbl="Concentration (M)", reg_rate=regular_rate)
        all_fit_results4p[(cyt, car)] = fit_res
        data_to_plot4p[(cyt, car)] = df_all_stats
all_fit_results4p = pd.concat(all_fit_results4p, names=("Cytokine", "CAR")).sort_index()
data_to_plot4p = pd.concat(data_to_plot4p, names=("Cytokine", "CAR")).sort_index()
all_fit_results4p

In [ ]:
# @title Plot all fit results
pep_order = ["N4", "A2", "Q4", "T4", "V4", "G4", "E1", "None"]
pep_order = [pep for pep in pep_order if pep in df_fit.index.get_level_values("Peptide").unique()]
palette = sns.color_palette(n_colors=len(pep_order))
markers = ["o", "s", "v"]
styles = ["-", "--", ":"]

# rows = cytokines, columns = CAR type
cytos, cars = df_fit.index.get_level_values("Cytokine").unique(), df_fit.index.get_level_values("CAR").unique()
#cytos = ["IL-2"]
fig, axes = plt.subplots(len(cytos), len(cars), sharey="row", sharex=True)
#axes = axes[None, :]
fig.set_size_inches(len(cars)*4.0, len(cytos)*3.5)
cytos_min_conc = df_clean.groupby("Cytokine").min()
pulse_min_conc = np.log10(np.min(df_fit.index.get_level_values("Concentration (M)").values))
for i, cyt in enumerate(cytos):
    for j, car in enumerate(cars):
        df_plot = data_to_plot4p.loc[(cyt, car)]
        fit_res = all_fit_results4p.loc[(cyt, car)]
        ax = axes[i, j]
        for p, pep in enumerate(pep_order):
            for r, rep in enumerate(fit_res.index.get_level_values("Spleen").unique()):
                x = df_plot.loc[(pep, rep), "log10 Concentration"].values
                xfit = np.linspace(x.min(), x.max(), 201)
                y = df_plot.loc[(pep, rep), "log10 Cytokine"].values
                pms = fit_res.loc[(pep, rep), "amplitude":"hill_power"].values
                yfit = hillFunction4p(xfit, pms)
                # Restore absolute cytokine scale (for y axis)
                # Error doesn't change: abs. scale is just adding a constant in log scale
                # so error bar is still y_log +- error. 
                y = y + np.log10(cytos_min_conc.loc[cyt].values)
                yfit = yfit + np.log10(cytos_min_conc.loc[cyt].values)

                # Restore absolute pulse scale (for x axis)
                x, xfit = x + pulse_min_conc, xfit + pulse_min_conc

                # Back to linear scale for plotting
                x, xfit = 10**x, 10**xfit
                y, yfit = 10**y, 10**yfit
                # Plot dose in uM, so multiply M doses by 1e6 
                lbl = (pep if r == 0 else None)
                ax.plot(x*1e6, y, marker=markers[r], mfc=palette[p], mec=palette[p], ls="none", ms=6)
                ax.plot(xfit*1e6, yfit, color=palette[p], label=lbl, ls=styles[r])
        ax.set(xscale="log", yscale="log")
# Label and legend as appropriate
for j in range(len(cars)):
    axes[-1, j].set_xlabel(r"Pulse ($\mu$M)")
    axes[0, j].set_title(cars[j])
for i in range(len(cytos)):
    axes[i, 0].set_ylabel("[{}] (nM)".format(cytos[i]))
leg = fig.legend(*axes[0, 0].get_legend_handles_labels(), loc="upper left", 
                 bbox_to_anchor=(0.99, 0.95), frameon=False)
fig.tight_layout()
if do_save_plots:
    fig.savefig("../figures/dose_response/ot1_ec50_dose_response_log_hill_fits.pdf", transparent=True, 
                bbox_inches="tight", bbox_extra_artists=(leg,))
plt.show()
plt.close()

In [ ]:
# Thesis version with only 2 cytokines, takes less space
# @title Plot all fit results
pep_order = ["N4", "A2", "Q4", "T4", "V4", "G4", "E1", "None"]
pep_order = [pep for pep in pep_order if pep in df_fit.index.get_level_values("Peptide").unique()]
palette = sns.color_palette(n_colors=len(pep_order))
markers = ["o", "s", "v"]
styles = ["-", "--", ":"]

# rows = cytokines, columns = CAR type
cytos, cars = df_fit.index.get_level_values("Cytokine").unique(), df_fit.index.get_level_values("CAR").unique()
cars = ["Mock"]
fig, axes = plt.subplots(1, len(cytos), sharex=True)
axes = axes[:, None]
fig.set_size_inches(len(cytos)*4.2, 4.0)
cytos_min_conc = df_clean.groupby("Cytokine").min()
pulse_min_conc = np.log10(np.min(df_fit.index.get_level_values("Concentration (M)").values))
for i, cyt in enumerate(cytos):
    for j, car in enumerate(cars):
        df_plot = data_to_plot4p.loc[(cyt, car)]
        fit_res = all_fit_results4p.loc[(cyt, car)]
        ax = axes[i, j]
        for p, pep in enumerate(pep_order):
            for r, rep in enumerate(fit_res.index.get_level_values("Spleen").unique()):
                x = df_plot.loc[(pep, rep), "log10 Concentration"].values
                xfit = np.linspace(x.min(), x.max(), 201)
                y = df_plot.loc[(pep, rep), "log10 Cytokine"].values
                pms = fit_res.loc[(pep, rep), "amplitude":"hill_power"].values
                yfit = hillFunction4p(xfit, pms)
                # Restore absolute cytokine scale (for y axis)
                # Error doesn't change: abs. scale is just adding a constant in log scale
                # so error bar is still y_log +- error. 
                y = y + np.log10(cytos_min_conc.loc[cyt].values)
                yfit = yfit + np.log10(cytos_min_conc.loc[cyt].values)

                # Restore absolute pulse scale (for x axis)
                x, xfit = x + pulse_min_conc, xfit + pulse_min_conc

                # Back to linear scale for plotting
                x, xfit = 10**x, 10**xfit
                y, yfit = 10**y, 10**yfit
                # Plot dose in uM, so multiply M doses by 1e6 
                lbl = (pep if r == 0 else None)
                ax.plot(x*1e6, y, marker=markers[r], mfc=palette[p], mec=palette[p], ls="none", ms=6)
                ax.plot(xfit*1e6, yfit, color=palette[p], label=lbl, ls=styles[r])
        ax.set(xscale="log", yscale="log")
# Label and legend as appropriate
for i in range(len(cytos)):
    cytolbl = "TNF" if cytos[i] == "TNFa" else cytos[i]
    axes[i, 0].set_xlabel(r"Pulse ($\mu$M)")
    axes[i, 0].set_title(cytolbl)
    axes[i, 0].set_ylabel("[{}] (nM)".format(cytolbl))
leg = fig.legend(*axes[0, 0].get_legend_handles_labels(), loc="upper left", 
                 bbox_to_anchor=(0.98, 0.95), frameon=False)
fig.tight_layout()
if do_save_plots:
    fig.savefig("../figures/dose_response/ot1_ec50_dose_response_phdthesis.pdf", transparent=True, 
                bbox_inches="tight", bbox_extra_artists=(leg,))
plt.show()
plt.close()

In [ ]:
#@title Plot all hill fit parameters
#Each spleen separately
plottingDf = all_fit_results4p.copy()

# Sort peptides
sort_fct = lambda x: pd.Index([pep_order.index(a) for a in x], name=x.name)
plottingDf = plottingDf.sort_index(level="Peptide", key=sort_fct)

# Absolute pulse concentration in uM: EC50
plottingDf["ec50"] = 10**(plottingDf["ec50"] + pulse_min_conc) * 1e6
plottingDf["amplitude"] = 10**plottingDf["amplitude"]
plottingDf["background"] = 10**plottingDf["background"]


plottingDf.columns.name = 'Statistic'

plottingDf = plottingDf.stack().to_frame('Value')
g = sns.catplot(data=plottingDf.reset_index(),x='Peptide',y='Value',row='Statistic',
                hue='CAR',kind='point',col='Cytokine',sharey="row",margin_titles=True, 
                hue_order=["Mock", "CAR_WT", "CAR_Mut"])
for i in range(g.axes.shape[0]):
    if all_fit_results4p.columns[i] in ["hill_power", "rsquared"]: continue
    for j in range(g.axes.shape[1]):
        g.axes[i, j].set_yscale("log")
plt.show()
plt.close()

# EC50s based on CD25
For comparison of cell lines and further model predictions. This part will generate `experimental_peptide_ec50s_blasts.h5`. 

In [ ]:
df_cd25 = pd.read_hdf(os.path.join(data_dir, "fullCD25EC50df.hdf"), key="df")
new_names = list(df_cd25.index.names)
new_names[new_names.index("Concentration_M")] = "Concentration (M)"
df_cd25.index = df_cd25.index.set_names(new_names)
df_cd25

In [ ]:
#@title Fit all dose response curves with sigmoid. Fit each spleen separately. 
df_fit = df_cd25.sort_index()  #np.log10(df_cd25)# / df_cd25.min())
# Different reg. rate for each parameter. Constrain amplitude a lot. 
# amplitude, ec50, background
regular_rate = 0.05

all_fit_results_cd25 = {}
data_to_plot_cd25 = {}
for tcr in df_cd25.index.get_level_values("TCR").unique():
    print("Fitting {}".format(tcr))
    fit_res, fit_data = fit_hill_cyto_each_peptide_replicate(df_fit.loc[tcr, "Percent_CD25+"], 
          conc_lbl="Concentration (M)", reg_rate=regular_rate)
    all_fit_results_cd25[tcr] = fit_res
    data_to_plot_cd25[tcr] = fit_data
all_fit_results_cd25 = pd.concat(all_fit_results_cd25, names=("TCR",))
data_to_plot_cd25 = pd.concat(data_to_plot_cd25, names=("TCR",))
all_fit_results_cd25 = all_fit_results_cd25.sort_index()
data_to_plot_cd25 = data_to_plot_cd25.sort_index()
all_fit_results_cd25

In [ ]:
# @title Plot all fit results
tcr_order = ["OT1", "NYESO", "HHAT"]
pep_orders = {
    "OT1": ["N4", "A2", "Q4", "T4", "V4", "G4", "E1"], 
    "NYESO": ["9V", "9C", "8S", "8K", "4A5P8K"], 
    "HHAT": ["p8F", "WT"]
}
palettes = {
    "OT1": sns.color_palette(n_colors=len(pep_orders["OT1"])), 
    "NYESO": sns.color_palette("Set2", n_colors=len(pep_orders["NYESO"])), 
    "HHAT": ["r", "b"]
}
markers = ["o", "s", "v"]
styles = ["-", "--", ":"]

# rows = cytokines, columns = CAR type
tcrs = df_fit.index.get_level_values("TCR").unique()
#cytos = ["IL-2"]
fig, axes = plt.subplots(1, len(tcrs), sharey=True, sharex=True)
axes = axes.flatten()

#fig.set_size_inches(6., 5.*len(tcrs))
fig.set_size_inches(5*len(tcrs), 5.0)
cd25_min_conc = df_cd25.min().values
pulse_min_conc = np.log10(np.min(df_cd25.index.get_level_values("Concentration (M)").values))
for i, tcr in enumerate(tcrs):
    df_plot = data_to_plot_cd25.loc[tcr]
    fit_res = all_fit_results_cd25.loc[tcr]
    ax = axes[i]
    palette = palettes[tcr]
    pep_order = pep_orders[tcr]
    for p, pep in enumerate(pep_order):
        for r, rep in enumerate(fit_res.index.get_level_values("Replicate").unique()):
            x = df_plot.loc[(pep, rep), "log10 Concentration"].values
            xfit = np.linspace(x.min(), x.max(), 201)
            y = df_plot.loc[(pep, rep), "log10 Cytokine"].values
            pms = fit_res.loc[(pep, rep), "amplitude":"hill_power"].values
            yfit = hillFunction4p(xfit, pms)
            # Restore absolute cytokine scale (for y axis)
            # Error doesn't change: abs. scale is just adding a constant in log scale
            # so error bar is still y_log +- error. 

            # Restore absolute pulse scale (for x axis)
            x, xfit = x + pulse_min_conc, xfit + pulse_min_conc

            # Back to linear scale for plotting
            x, xfit = 10**x, 10**xfit
            # Plot dose in uM, so multiply M doses by 1e6 
            lbl = (pep if r == 0 else None)
            ax.plot(x*1e6, y, marker=markers[r], mfc=palette[p], mec=palette[p], ls="none", ms=6)
            ax.plot(xfit*1e6, yfit, color=palette[p], label=lbl, ls=styles[r])
    ax.set(xscale="log")
# Label and legend as appropriate
axes[0].set_ylabel("% CD25+")
for j in range(len(tcrs)):
    axes[j].set_xlabel(r"Pulse ($\mu$M)")
    axes[j].set_xlim([5e-8, 3e1])
    axes[j].set_title(tcrs[j])
    axes[j].legend()#loc="upper left", bbox_to_anchor=(0.98, 0.95))
fig.tight_layout()
#fig.savefig("../figures/dose_response/cd25_dose_response_fits.pdf", transparent=True, bbox_inches="tight")
plt.show()
plt.close()

# Collect all fitted EC50s with various methods

In [19]:
#@title Function to scale back parameters to absolute concentrations
def put_back_absolute_scales(fitres, df_dat, ylog=True):
    # Rename "Cytokine" to "Marker", if relevant
    # Put back absolute lower bounds in log scale
    rename_dict = {"Cytokine":"Marker", "Spleen":"Replicate"}
    fitres.index = fitres.index.rename([rename_dict.get(a, a) for a in fitres.index.names])
    for cyt in fitres.index.get_level_values("Marker").unique():
        for car in fitres.index.get_level_values("CAR").unique():
            min_conc = float(np.min(df_dat.index.get_level_values("Concentration (M)").values))
            fitres.loc[(cyt, car), "ec50"] = fitres.loc[(cyt, car), "ec50"].add(np.log10(min_conc)).values
            
        min_cyto = float(df_dat.loc[cyt].min().iat[0])
        fitres.loc[cyt, "background"] = fitres.loc[cyt, "background"].add(np.log10(min_cyto)).values
    
    # Convert to linear scale amplitude, ec50, background. 
    # amplitude = fold-change. Linear-scale response ranges from 10^b to 10^(a+b), 
    # so 10^a = fold-change wrt background
    if ylog:
        fitres.loc[:, ["amplitude", "ec50", "background"]] = (
                    10**fitres.loc[:, ["amplitude", "ec50", "background"]])
    else:
        fitres.loc[:, "ec50"] = 10**fitres.loc[:, "ec50"]
    
    # If there are parameter covariances, convert to lower and upper bounds on param values
    # If error on log(x) is s, then x_up = x * 10**s, x_low = x * 10**(-s)
    for col in fitres.columns:
        if col.startswith("cov_"):
            pname = col[4:]
            s = np.sqrt(fitres.loc[:, col])
            if ylog:
                fitres["lower_"+pname] = fitres[pname] * 10**(-s)
                fitres["upper_"+pname] = fitres[pname] * 10**s
            else:
                fitres["lower_"+pname] = fitres[pname] - s
                fitres["upper_"+pname] = fitres[pname] + s
            fitres = fitres.drop(col, axis=1)
    return fitres

In [20]:
#@title Put back absolute scales. EC50 is in M and background is in nM
final_results_hill_4p = put_back_absolute_scales(all_fit_results4p.copy(), df_clean)

# Add a TCR level
final_results_hill_4p = pd.concat({"OT1":final_results_hill_4p}, names=["TCR"])

all_cd25_2 = pd.concat([all_fit_results_cd25.copy().sort_index()], keys=[("CD25", "CAR_WT")], names=["Marker", "CAR"])
df_cd25_data = pd.concat([df_cd25.copy().sort_index()], keys=[("CD25",)], names=["Marker"])
final_results_cd25 = put_back_absolute_scales(all_cd25_2, df_cd25_data, ylog=False)
final_results_cd25 = final_results_cd25.reorder_levels(final_results_hill_4p.index.names, axis=0)

In [ ]:
# Make a final plot of the EC50s from all methods. They should all agree pretty well. 
hill_indiv_dummy = final_results_hill_4p.copy()
display(hill_indiv_dummy)
hill_indiv_dummy = hill_indiv_dummy.reset_index().set_index(["TCR", "Marker", "CAR", "Peptide", "Replicate"])
all_ec50_df = pd.concat({
    "Hill_individual": final_results_hill_4p,
    "CD25fit": final_results_cd25
}, names=["Method"])
all_ec50_df = all_ec50_df["ec50"]
all_ec50_df

# Conclusions
The methods I used are pretty consistent, the CAR makes only a small difference, while the cytokine chosen makes the biggest difference.

The CD25 EC50 is also very similar. 

In [23]:
all_ec50_df.name = "ec50_M"
if do_save_outputs:
    all_ec50_df.to_hdf("../data/dose_response/experimental_peptide_ec50s_blasts.h5", key="df")